In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import random

In [2]:
import os

class CustomDataset(Dataset):
    def __init__(self, ndata):
        self.X = []
        self.Y = []
        for i in range(ndata):
            x = random.random()*100
            self.X.append(x)
            self.Y.append(x*2)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [3]:
# Download training data from open datasets.
training_data = CustomDataset(ndata=1000)
print(training_data[0])

# Download test data from open datasets.
test_data = CustomDataset(ndata=100)
print(len(test_data))

(50.19668129095615, 100.3933625819123)
100


In [4]:
train_dataloader = DataLoader(training_data, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=True)

In [5]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        #self.flatten = nn.Flatten()
        self.l1 = nn.Linear(1, 64)
        self.r1 = nn.ReLU()
        self.l2 = nn.Linear(64,64)
        self.r2 = nn.ReLU()
        self.l3 = nn.Linear(64,1)
        

    def forward(self, x):
        x = self.l1(x)
        x = self.r1(x)
        x = self.l2(x)
        x = self.r2(x)
        x = self.l3(x)
        return x

model = NeuralNetwork().double().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (l1): Linear(in_features=1, out_features=64, bias=True)
  (r1): ReLU()
  (l2): Linear(in_features=64, out_features=64, bias=True)
  (r2): ReLU()
  (l3): Linear(in_features=64, out_features=1, bias=True)
)


In [16]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

In [17]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        X = X.type(torch.DoubleTensor)
        y = y.type(torch.DoubleTensor)

        pred = model(X)
        loss = loss_fn(pred, y)

        #print(loss.item())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        if batch % 100 == 0:
            loss_val, current = loss.item(), batch * len(X)
            print(f"loss: {loss.item():>7f}  [{current:>5d}/{size:>5d}]")

In [18]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            if abs(pred - y) < 10**(-5) :
                correct += 1
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [19]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 29074239229885233930796155147386880.000000  [    0/ 1000]
loss: 27934110251896885715380228654628864.000000  [  100/ 1000]
loss: 26838690752845366589561446651133952.000000  [  200/ 1000]
loss: 25786227477137932927443646843715584.000000  [  300/ 1000]
loss: 24775035922056324940795020316770304.000000  [  400/ 1000]
loss: 23803497641653033277719265190871040.000000  [  500/ 1000]
loss: 22870057656374604276133137347510272.000000  [  600/ 1000]
loss: 21973221964264989783583827630751744.000000  [  700/ 1000]
loss: 21111555149764947497260910962016256.000000  [  800/ 1000]
loss: 20283678086281752395465967047540736.000000  [  900/ 1000]
Test Error: 
 Accuracy: 0.0%, Avg loss: 19488265728850760942330478631845888.000000 

Epoch 2
-------------------------------
loss: 19488265728850749413115432563376128.000000  [    0/ 1000]
loss: 18724044993356497936595512677367808.000000  [  100/ 1000]
loss: 17989792718919005142972387416866816.000000  [  200/ 1000]
los

In [20]:
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

TypeError: linear(): argument 'input' (position 1) must be Tensor, not float